<a href="https://colab.research.google.com/github/juanazorzolo/otraprueba/blob/main/TP1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRABAJO PRÁCTICO 1 PROCESAMIENTO DEL LENGUAJE NATURAL TUIA 2024**



INTEGRANTES:

Herrera Morena (H-1187/8)

Zorzolo Rubio Juana (Z-1217/3)

# Importaciones

In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install sentiment-analysis-spanish
!pip install transformers
!python -m spacy download en_core_web_md
!pip install sacremoses
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import spacy
import unicodedata
import sacremoses
import time
from ipywidgets import Text, Button, Output, VBox, HTML, Label, HBox
from IPython.display import display
from tabulate import tabulate
from bs4 import BeautifulSoup
from gliner import GLiNER
from transformers import BertTokenizer, BertForSequenceClassification, pipeline, MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Carga de datos

Decidimos realizar modificaciones en los nombres de columnas de los conjuntos de datos para simplificar su manejo. Este cambio facilita el manejo de variables y mejora el flujo de trabajo al parsear información.

## Juegos

In [3]:
!gdown "1v1jO8s3g1V4wjyNp58zF3AVwJibS1kll" --output "bgg_database.csv"

Downloading...
From: https://drive.google.com/uc?id=1v1jO8s3g1V4wjyNp58zF3AVwJibS1kll
To: /content/bgg_database.csv
100% 1.83M/1.83M [00:00<00:00, 158MB/s]


In [4]:
df_juegos = pd.read_csv('bgg_database.csv')
df_juegos.head()

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.60,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']"
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight..."
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.335,8.54,44728.0,"In Ark Nova, you will plan and design a modern...",2021,1,4,90,150,14,3.7653,"[{'min': 2, 'max': 2}]",['Mathias Wigge'],"['Action Queue', 'End Game Bonuses', 'Grid Cov...","['Animals', 'Economic', 'Environmental']"
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.240,8.60,24148.0,Twilight Imperium (Fourth Edition) is a game o...,2017,3,6,240,480,14,4.3173,"[{'min': 6, 'max': 6}]","['Dane Beltrami', 'Corey Konieczka', 'Christia...","['Action Drafting', 'Area-Impulse', 'Dice Roll...","['Civilization', 'Economic', 'Exploration', 'N..."


In [5]:
df_juegos = df_juegos.rename(columns={'game_name': 'name'})
df_juegos = df_juegos.rename(columns={'game_href': 'link'})

# Filtrar columnas necesarias
df_juegos = df_juegos[['name', 'description', 'link']]
# en un futuro se podrian utilizar las de multijugadores y categorias para mejores versiones del sistema

In [6]:
df_juegos.head()

,name,description,link
0,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,https://boardgamegeek.com/boardgame/224517/bra...
1,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,https://boardgamegeek.com/boardgame/161936/pan...
2,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,https://boardgamegeek.com/boardgame/174430/glo...
3,Ark Nova,"In Ark Nova, you will plan and design a modern...",https://boardgamegeek.com/boardgame/342942/ark...
4,Twilight Imperium: Fourth Edition,Twilight Imperium (Fourth Edition) is a game o...,https://boardgamegeek.com/boardgame/233078/twi...


## Películas

In [7]:
!gdown "1NoEtyFXuCN72QSIxR8amIcrH3X4NKy3h" --output "IMDB-Movie-Data.csv"

Downloading...
From: https://drive.google.com/uc?id=1NoEtyFXuCN72QSIxR8amIcrH3X4NKy3h
To: /content/IMDB-Movie-Data.csv
100% 309k/309k [00:00<00:00, 101MB/s]


In [8]:
df_pelis = pd.read_csv('IMDB-Movie-Data.csv')
df_pelis.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40


In [9]:
df_pelis = df_pelis.rename(columns={'Title': 'name'})
df_pelis = df_pelis.rename(columns={'Description': 'description'})
df_pelis = df_pelis.rename(columns={'Director': 'director/author'})
df_pelis = df_pelis[['name', 'director/author', 'description']]

In [10]:
df_pelis.head()

,name,director/author,description
0,Guardians of the Galaxy,James Gunn,A group of intergalactic criminals are forced ...
1,Prometheus,Ridley Scott,"Following clues to the origin of mankind, a te..."
2,Split,M. Night Shyamalan,Three girls are kidnapped by a man with a diag...
3,Sing,Christophe Lourdelet,"In a city of humanoid animals, a hustling thea..."
4,Suicide Squad,David Ayer,A secret government agency recruits some of th...


## Libros

### Web Scraping

In [11]:
"""url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

books = []
contador = 0

for indice, item_libro in enumerate(soup.find_all('li')):
  enlace_libro = item_libro.find('a')

  # Filtra solo enlaces que contengan "/ebooks/" seguido de un número
  if enlace_libro and re.search(r'/ebooks/\d+$', enlace_libro['href']):
    # Extraer el título y el autor del texto completo en el enlace
    texto_completo = enlace_libro.text.strip()

    # Dividir el texto en título y autor
    if " de " in texto_completo:
      titulo, autor = texto_completo.split(" de ", 1)
    elif " by " in texto_completo:
      titulo, autor = texto_completo.split(" by ", 1)
    else:
      titulo = texto_completo
      autor = "Desconocido"

    # Limpiar los textos de paréntesis y espacios adicionales
    titulo = titulo.strip()
    autor = autor.split("(")[0].strip()  # Quitar cualquier número entre paréntesis

    link = 'https://www.gutenberg.org' + enlace_libro['href']

    # Obtener el género y la descripción del libro desde su página individual
    genero = "Desconocido"
    descripcion = "Desconocida"
    book_response = requests.get(link)
    book_soup = BeautifulSoup(book_response.text, 'html.parser')

    # Buscar el género
    subject_section = book_soup.find('th', string=re.compile("Subject|Categorías", re.IGNORECASE))
    if subject_section:
      generos = subject_section.find_next('td').find_all('a')
      genero = ', '.join([g.text for g in generos])

    # Buscar la descripción
    seccion_descripcion = book_soup.find('th', string=re.compile("Summary", re.IGNORECASE))
    if seccion_descripcion:
      descripcion = seccion_descripcion.find_next('td').text.strip()  # Directamente desde el <td>

    books.append({
      'title': titulo,
      'author': autor,
      'link': link,
      'genre': genero,
      'description': descripcion
    })

    contador += 1
    if contador >= 1000:
      break

    # Pausa para evitar sobrecargar el servidor
    time.sleep(0.01)

# Crear el DataFrame y guardar en CSV
df = pd.DataFrame(books)
df.to_csv('gutenberg_books.csv', index=False)"""

'url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.text, \'html.parser\')\n\nbooks = []\ncontador = 0\n\nfor indice, item_libro in enumerate(soup.find_all(\'li\')):\n  enlace_libro = item_libro.find(\'a\')\n\n  # Filtra solo enlaces que contengan "/ebooks/" seguido de un número\n  if enlace_libro and re.search(r\'/ebooks/\\d+$\', enlace_libro[\'href\']):\n    # Extraer el título y el autor del texto completo en el enlace\n    texto_completo = enlace_libro.text.strip()\n\n    # Dividir el texto en título y autor\n    if " de " in texto_completo:\n      titulo, autor = texto_completo.split(" de ", 1)\n    elif " by " in texto_completo:\n      titulo, autor = texto_completo.split(" by ", 1)\n    else:\n      titulo = texto_completo\n      autor = "Desconocido"\n\n    # Limpiar los textos de paréntesis y espacios adicionales\n    titulo = titulo.strip()\n    autor = autor.split("(")[0].strip()  # Quitar cual

In [12]:
!gdown "1tkpREPES_9wqQ4AaYSwpY1dePBeLaFtQ" --output "gutenberg_books.csv"

Downloading...
From: https://drive.google.com/uc?id=1tkpREPES_9wqQ4AaYSwpY1dePBeLaFtQ
To: /content/gutenberg_books.csv
100% 1.24M/1.24M [00:00<00:00, 124MB/s]


In [13]:
df_libros = pd.read_csv('gutenberg_books.csv')
df_libros.head()

,title,author,link,genre,description
0,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,https://www.gutenberg.org/ebooks/84,\nScience fiction\n,"""Frankenstein; Or, The Modern Prometheus"" by M..."
1,呻吟語,Kun Lü,https://www.gutenberg.org/ebooks/25558,\nConduct of life\n,"""呻吟語"" by Kun Lü is a philosophical treatise wr..."
2,Pride and Prejudice,Jane Austen,https://www.gutenberg.org/ebooks/1342,\nEngland -- Fiction\n,"""Pride and Prejudice"" by Jane Austen is a clas..."
3,"Moby Dick; Or, The Whale",Herman Melville,https://www.gutenberg.org/ebooks/2701,\nWhaling -- Fiction\n,"""Moby Dick; Or, The Whale"" by Herman Melville ..."
4,The Scarlet Letter,Nathaniel Hawthorne,https://www.gutenberg.org/ebooks/25344,\nAdultery -- Fiction\n,"""The Scarlet Letter"" by Nathaniel Hawthorne is..."


In [14]:
df_libros['genre'] = df_libros['genre'].str.replace(r'\n', '', regex=True).str.strip()
df_libros = df_libros.rename(columns={'title': 'name'})
df_libros = df_libros.rename(columns={'author': 'director/author'})
df_libros = df_libros[['name', 'director/author', 'link', 'description']]

In [15]:
df_libros.head()

,name,director/author,link,description
0,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,https://www.gutenberg.org/ebooks/84,"""Frankenstein; Or, The Modern Prometheus"" by M..."
1,呻吟語,Kun Lü,https://www.gutenberg.org/ebooks/25558,"""呻吟語"" by Kun Lü is a philosophical treatise wr..."
2,Pride and Prejudice,Jane Austen,https://www.gutenberg.org/ebooks/1342,"""Pride and Prejudice"" by Jane Austen is a clas..."
3,"Moby Dick; Or, The Whale",Herman Melville,https://www.gutenberg.org/ebooks/2701,"""Moby Dick; Or, The Whale"" by Herman Melville ..."
4,The Scarlet Letter,Nathaniel Hawthorne,https://www.gutenberg.org/ebooks/25344,"""The Scarlet Letter"" by Nathaniel Hawthorne is..."


# Clasificador

En base al sentimiento ingresado por el usuario, lo categoriza en un rango de 1 a 5 para definir si su estado es 'alegre', 'melancólico' o 'ni fu ni fa'.

In [16]:
# Cargar el tokenizador y el modelo BERT
tokenizador_sentimiento = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
modelo_sentimiento = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [17]:
warnings.filterwarnings("ignore", category=FutureWarning)

def analizar_sentimiento_usuario(frase_usuario):
  # Crear un pipeline de clasificación
  nlp = pipeline("sentiment-analysis", model=modelo_sentimiento, tokenizer=tokenizador_sentimiento)

  # Obtener el resultado del análisis de sentimiento
  resultado_sentimiento = nlp(frase_usuario)

  # Extraer el número de estrellas de la etiqueta
  label = resultado_sentimiento[0]['label']
  nro_estrellas = int(label.split()[0])

  print(f"Frase del usuario: '{frase_usuario}'")
  print(f"  Sentimiento: {label}")

  return nro_estrellas

def determinar_estado_animo(nro_estrellas):
  if nro_estrellas < 3:
    print("Estado de ánimo del usuario: Melancólico")
  elif nro_estrellas == 5:
    print("Estado de ánimo del usuario: Alegre")
  else:
    print("Estado de ánimo del usuario: Ni fu ni fa")

# Parseo de texto

En base a la descripción del libro, película o juego, la pasa a minúscula, elimina acentos y carcateres especiales y reemplaza cualquier signo que no sea alfanumérico por un espacio.

In [18]:
# Cargar el modelo de lenguaje
nlp = spacy.load("en_core_web_md")

In [19]:
# Cargar el modelo NER con la libreria gliner
modelo_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
# Cargar el modelo para calcular embeddings
modelo_embeddings = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
def limpiar_texto(texto):
  texto = texto.lower()
  texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
  texto = re.sub(r'\W+', ' ', texto)

  if len(texto) > 10000:
    texto = texto[:10000]

  return texto

def obtener_entidades(texto):
  texto = limpiar_texto(texto)

  # Entidades
  labels = ["book", "location", "event", "person", "organization", "date", "time",
            "quantity", "product", "work of art", "language", "currency", "law", "facility"]

  entidades = modelo_gliner.predict_entities(texto, labels, threshold=0.4)
  return [(entidad["text"], entidad["label"]) for entidad in entidades]

def procesar_descripciones(data):
  # Obtener entidades
  data['Description_entities'] = data['description'].apply(obtener_entidades)

  # Obtener embeddings
  embeddings = []
  for indice, fila in data.iterrows():
    emb_descripcion = modelo_embeddings.encode(fila['description'])
    embeddings.append(emb_descripcion)

  data['Description_embeddings'] = embeddings

  return data

In [22]:
df_pelis = procesar_descripciones(df_pelis)
df_juegos = procesar_descripciones(df_juegos)
df_libros = procesar_descripciones(df_libros)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 446 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 394 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 408 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 433 has been truncated to 384
  warnings.warn(f"Sentence of le

# Traducción NMT

In [23]:
# Cargar el modelo NMT y el tokenizador
tokenizador_traduccion = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-es-en')
modelo_traduccion = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-es-en')

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [24]:
def traducir_procesar_preferencia(preferencia):
  print("Preferencia ingresada:", preferencia)
  print("")

  preferencia = limpiar_texto(preferencia)
  print("  Frase parseada:", preferencia)
  print("")
  # Traducir la preferencia del usuario al inglés
  inputs = tokenizador_traduccion(preferencia, return_tensors="pt")
  outputs = modelo_traduccion.generate(**inputs, max_length=500)
  preferencia_ingles = tokenizador_traduccion.decode(outputs[0], skip_special_tokens=True)
  print("  Traducción:", preferencia_ingles)
  print("")
  # Obtener entidades
  entidades = obtener_entidades(preferencia_ingles)
  print("  Entidades encontradas:", entidades)

  # Obtener embeddings
  emb_preferencia = modelo_embeddings.encode(preferencia_ingles)

  return emb_preferencia, entidades

# Recomendación de opciones

Se identifica la cantidad de similitudes entre las entidades de la preferencia ingresada y las entidades de las descripciones de las actividades.

En base los embeddings de la preferencia y las descripciones de peliculas, libros y juegos, se calcula la distancia con la similitud del coseno.

Finalmente, se eligen las 3 recomendaciones, se priorizan las que tiene la mayor cantidad de coincidencias de entidades, o, en su defecto cuando no haya coincidencias, las mas cercanas en términos de similitud de embeddings

In [25]:
def obtener_top_3(data, emb_preferencia, entidades_preferencia):
  lista_resultados = []

  for indice, fila in data.iterrows():
    # Obtener el embedding
    emb_descripcion = fila['Description_embeddings']
    # Calcular la distancia entre la preferencia y la descripción
    similitud = np.dot(emb_preferencia, emb_descripcion) / (np.linalg.norm(emb_preferencia) * np.linalg.norm(emb_descripcion))

    # Obtener las entidades
    entidades_descripcion = fila['Description_entities']
    # Contar coincidencias de entidades entre la preferencia y la descripción
    coincidencias = sum(1 for entidad_preferencia in entidades_preferencia if any(entidad_preferencia == entidad_descripcion for entidad_descripcion in entidades_descripcion))

    resultado = {
      'name': fila['name'],
      'matches': coincidencias,
      'similarity': similitud,
                }

    # Agregar director/autor y link
    if 'director/author' in fila:
      resultado['director/author'] = fila['director/author']
    if 'link' in fila:
      resultado['link'] = fila['link']

    lista_resultados.append(resultado)

  data_con_resultados = pd.DataFrame(lista_resultados)

  # Ordenar el data por coincidencias y similitud, y seleccionar los 3 principales
  top_3_data = data_con_resultados.sort_values(by=['matches', 'similarity'], ascending=[False, False]).head(3)

  columnas_deseadas = ['name', 'director/author', 'link', 'matches', 'similarity']
  columnas_disponibles = [col for col in columnas_deseadas if col in top_3_data.columns]

  data_sugerencias = top_3_data[columnas_disponibles].reset_index(drop=True)

  return data_sugerencias

def mostrar_top_tres(top_data):
  top_data['similarity'] = top_data['similarity'].round(4)
  print(tabulate(top_data, headers='keys', tablefmt='grid', showindex=False))

# Programa principal

Se crea la interfaz para interactuar con el usuario y teniendo en cuenta la frase ingresada como preferencia, el programa ofrece recomendaciones pertinentes entre películas, juegos de mesa o libros.

In [26]:
def programa_principal(b):
  with output:
    output.clear_output()

    # Obtener el estado de animo del usuario
    frase_usuario = frase_input.value
    nro_estrellas = analizar_sentimiento_usuario(frase_usuario)
    determinar_estado_animo(nro_estrellas)

    print("-" * 50)

    # Obtener la preferencia del usuario
    preferencia = preferencia_input.value
    emb_preferencia, entidades_preferencia = traducir_procesar_preferencia(preferencia)

    # Obtener las recomendaciones
    top_juegos = obtener_top_3(df_juegos, emb_preferencia, entidades_preferencia)
    top_pelis = obtener_top_3(df_pelis, emb_preferencia, entidades_preferencia)
    top_libros = obtener_top_3(df_libros, emb_preferencia, entidades_preferencia)

    print("\nTop 3 juegos sugeridos:")
    mostrar_top_tres(top_juegos)

    print("\nTop 3 películas sugeridos:")
    mostrar_top_tres(top_pelis)

    print("\nTop 3 libros sugeridos:")
    mostrar_top_tres(top_libros)

# Verificar si ambos campos están completos
def verificar_y_ejecutar(b):
  mensaje_error.value = ""
  if not frase_input.value.strip() or not preferencia_input.value.strip():
    mensaje_error.value = "<span style='color: red;'>Por favor, completar todos los campos.</span>"
  else:
    programa_principal(boton_recomendar)

In [27]:
# Definir widgets de entrada
frase_label = Label(value="¿Cómo te sentís hoy? :")
frase_input = Text(placeholder='Ejemplo: increíble', layout={'width': '60%', 'height': '10'})

preferencia_label = Label(value="¿Tenés alguna temática preferida? :")
preferencia_input = Text(placeholder='Ejemplo: comedia romántica', layout={'width': '60%', 'height': '10'})

# Botón y salida
boton_recomendar = Button(description="Obtener Recomendaciones", layout={'width': '20%'})
output = Output()
mensaje_error = HTML(value="")

# Conectar el evento de teclado al botón
frase_input.on_submit(verificar_y_ejecutar)
preferencia_input.on_submit(verificar_y_ejecutar)

# Mostrar los widgets
display(VBox([frase_label, frase_input, preferencia_label, preferencia_input, boton_recomendar, mensaje_error, output]))